#IMPORTS

In [ ]:
!pip install tensorflow==2.10.1  -q gwpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 102.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependen

In [ ]:
!pip uninstall matplotlib
!pip install matplotlib==3.1.3

Found existing installation: matplotlib 3.7.1
Uninstalling matplotlib-3.7.1:
  Would remove:
    /usr/local/lib/python3.9/dist-packages/matplotlib-3.7.1-py3.9-nspkg.pth
    /usr/local/lib/python3.9/dist-packages/matplotlib-3.7.1.dist-info/*
    /usr/local/lib/python3.9/dist-packages/matplotlib/*
    /usr/local/lib/python3.9/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.9/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.9/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.9/dist-packages/pylab.py
Proceed (Y/n)? 

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from google.colab import drive
import tensorflow as tf
import numpy as np
from typing import Tuple
import albumentations as A
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as img
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array,array_to_img
from sklearn.metrics import confusion_matrix
from keras.callbacks import *
import random
from keras import backend as K
from PIL import Image
from skimage.measure import label as label_fn
import cv2
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
import albumentations as A
from itertools import combinations
import json
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn import svm
from skimage import filters
import scipy

In [ ]:
!pip install lime

In [ ]:
import lime
from lime import lime_image
from lime import submodular_pick
from skimage.segmentation import mark_boundaries

#MOUNT DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/AAIB
PATH = '/content/drive/MyDrive/tuberculosis-pneumonia-classification'

In [ ]:
SEED = 4224
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

labels_path = 'data/labels_train_clean.csv'
all_data_no_duplicates_path = 'data/train_all_no_duplicates'
clean_data_path = 'data/train_clean/'
noisy_data_path = 'data/train_noisy/'

train_percentage = 0.8
validation_percentage = 0.15
test_percentage = 0.2
img_size = (224,224)
batch_size = 16

sns.set_style("ticks")
sns.despine()
sns.set_context('paper')

#DATA GENERATOR

In [ ]:
class CustomGenerator(tf.keras.utils.Sequence):
  """
    CustomGenerator inheriting from tf.keras.utils.Sequence.

    We have to implement 3 main methods:
      - __init__: save dataset params like directory, filenames, etc.
      - __len__: return the total number of samples in the dataset (number of batches)
      - __getitem__: return a single batch of paired images masks
  """

  def __init__(self, 
               dataframe, # dataframe of the dataset  
               base_path,
               preprocessing_function=None, # Preprocessing function (e.g., the one used for transfer learning)
               batch_size=16, # Batch size
               out_shape = (100,100),
               shuffle=False,
               categorical = True,
               augment = False,
               seed = SEED,
               flow_from_directory = True,
               preprocess_input = False):
    
    # Get all filenames
    if isinstance(base_path, Tuple):
      self.filenames = []
      for p in base_path:

        paths = self.folderToPaths(p, full_path = False)

        for pa in paths:
          if pa in set(dataframe.file):
            self.filenames.append(os.path.join(p, pa))


    else:
        self.filenames = [os.path.join(base_path, img_path) for img_path in list(dataframe.file)]

    self.labels = tfk.utils.to_categorical(list(dataframe.label)) if categorical else list(dataframe.label)

    # Set indices list in [0, len(subset_filenames)]
    self.indices = np.arange(len(self.filenames))

    # Save dataset parameters as class attributes
    self.base_path = base_path
    self.preprocessing_function = preprocessing_function
    self.out_shape = out_shape
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.augment = augment
    self.seed = seed
    self.flow_from_directory =flow_from_directory
    self.data_augmentation = A.Compose([
    A.RandomBrightnessContrast(brightness_limit = 0.05, contrast_limit=0.05, p=0.5),
    A.ShiftScaleRotate(p = 0.8, rotate_limit = 20, scale_limit = 0.3, border_mode =  cv2.BORDER_CONSTANT, value = 0),
    A.CLAHE(p=0.2)
    ])
    self.preprocess_input = preprocess_input

    if not self.flow_from_directory:
      self.images = self.load_all_imgs()

  def augmentation(self, images):
    return self.data_augmentation(image = images)


  def __filterNoisyOnClahe(self, image):
    clahe = cv2.createCLAHE(clipLimit = 300, tileGridSize = (50, 50))
    im1 = cv2.resize(image, (400, 400))
    im1 = scipy.ndimage.gaussian_laplace(im1, sigma = 6)
    im1 = clahe.apply(im1)
    var1 = np.var(im1)
    if var1 > 800:
      image= cv2.medianBlur(image, ksize=5)
      return scipy.ndimage.uniform_filter(image, size=3)
    else:
      return image


  def __sharpenImage(self, image):
    sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

    sharpened = cv2.filter2D(image, -1, sharpen_kernel)

    return sharpened

  def __invert_image(self, img):

    otsu_thresh = filters.threshold_otsu(img)
    masked_image = (img > otsu_thresh) * 1.0
    valsROI1, _ = np.histogram(masked_image[220:244, 220:244], bins=2, range=(0, 1))
    valsROI2, _ = np.histogram(masked_image[0:25, 0:25], bins=2, range=(0, 1))
    valsROI3, _ = np.histogram(masked_image[0:25, 220:244], bins=2, range=(0, 1))
    valsROI4, _ = np.histogram(masked_image[220:244, 0:25], bins=2, range=(0, 1))
    
    valsTot = valsROI1 + valsROI2 + valsROI3 + valsROI4
    
    labels = label_fn(masked_image)

    if len(np.unique(labels)) < 100:
        if valsTot[0] > valsTot[1]:
            return img
        else:
            return 255 - img
    return img

  def __filterBlurred(self, image):
    clahe = cv2.createCLAHE(clipLimit = 1.8, tileGridSize = (4, 4))
    minThresh = 2
    im1 = cv2.resize(image, (400, 400))
    im1 = scipy.ndimage.gaussian_laplace(im1, sigma = 2)
    im1 = clahe.apply(im1)
    _, im1 = cv2.threshold(im1, minThresh, 255, cv2.THRESH_BINARY)
    var1 = np.var(im1)
    if var1 < 5:
        return self.__sharpenImage(image)
    else:
        return self.__sharpenImage(image)

  def __filterUnderexposed(self, image):
    im1 = cv2.resize(image, (400, 400))
    mean1 = np.mean(im1)
    if mean1 < 71:
      clahe = cv2.createCLAHE(clipLimit = 2, tileGridSize = (2, 2))
      image = clahe.apply(image)
      return image
    else:
      return image


  def preprocess(self, image):

    image = self.__invert_image(image)
    image = self.__filterUnderexposed(image)
    image = self.__filterNoisyOnClahe(image)
    image = self.__filterBlurred(image)
   

    return image

  def __len__(self):
    # Return the length of the dataset (number of batches)
    # that is given by #images // batch_size
    return len(self.filenames) // self.batch_size

  def on_epoch_start(self):
    # Shuffle indices after each epoch
    if self.shuffle == True:
        np.random.shuffle(self.indices)

  def load_all_imgs(self):
      images = []
      for f in self.filenames:
        image = cv2.imread(f, 0)
        image = cv2.resize(image, (self.out_shape))
        if self.preprocess_input:
          image = self.preprocess(image)
        images.append(image)

      return np.array(images)

  def get_image_and_label(self, index):

    if not self.flow_from_directory:
      image = self.images[index]
      if self.augment:
        image = self.augmentation(image)
      image = np.squeeze(image)
      image = np.stack([image, image, image], axis = -1)
      curr_label = self.labels[index]
    else:
      curr_filename = self.filenames[index] # Get filename at index
      curr_label = self.labels[index]
      image = cv2.imread(curr_filename, 0)
      image = cv2.resize(image, (self.out_shape))
      if self.preprocess_input:
        image = self.preprocess(image)

      if self.augment:
        image = self.augmentation(image)['image']

      image = np.stack([image, image, image], axis = -1)


    return image, curr_label

  def __getitem__(self, index):
    # In this function we generate a batch (of size self.batch_size) of images and corresponding masks
    
    # Get 'self.batch_size' indices
    current_indices = self.indices[index*self.batch_size:(index*self.batch_size)+self.batch_size]

    """if len(current_indices) == 0:
      current_indices = self.indices[len(self.indices)-self.batch_size:len(self.indices)]"""

    # Init lists that will contain images and masks
    batch_images = []
    batch_labels = []

    # Cycle over the indices
    for idx in current_indices:
      # Get single image/mask at index 'idx'
      image, label = self.get_image_and_label(idx)

      # Apply the preprocessing function
      if self.preprocessing_function is not None:
        image = self.preprocessing_function(image)

      # Append both image and mask (with added batch dimension) to the corresponding batch lists
      batch_images.append(np.expand_dims(image, 0))
      batch_labels.append(label)
     
    # Finally, obtain a final batch by concatenating all the images over the batch dimension
    batch_images = np.concatenate(batch_images, axis=0)
    batch_labels = np.array(batch_labels)

    return batch_images, batch_labels


  def folderToPaths(
        self,
        full_img_dir,
        full_path = True
):

    x_paths_list = []

    full_img_dir = full_img_dir

    for full in os.listdir(full_img_dir):
         if full_path:
            x_paths_list.append(os.path.join(full_img_dir, full))
         else:
          x_paths_list.append(full)
    
    x_paths_list.sort()
    return x_paths_list

#DATA LOADING

In [ ]:
def encode(x):
  if x == 'N':
    return 0
  elif x == 'P':
    return 1
  else:
    return 2

In [ ]:
def folderToPaths(
        full_img_dir,
):

    x_paths_list = []

    full_img_dir = full_img_dir

    for full in os.listdir(full_img_dir):
         x_paths_list.append(os.path.join(full_img_dir, full))
    
    x_paths_list.sort()
    return x_paths_list

In [ ]:
labelsDF = pd.read_csv(labels_path)
display(labelsDF.head(20))

In [ ]:
labelsDF.label = labelsDF.label.apply(lambda x: encode(x))
display(labelsDF.head(20))

In [ ]:
all_data_no_duplicates_path_list = folderToPaths(full_img_dir = all_data_no_duplicates_path)

In [ ]:
train_val, test = train_test_split(labelsDF, test_size = test_percentage, shuffle = True, stratify = labelsDF.label, random_state=SEED)

In [ ]:
test_gen = CustomGenerator(dataframe = test, base_path = 'data/train_all_no_duplicates', batch_size = batch_size, out_shape = img_size, shuffle = True, flow_from_directory=True, preprocess_input = True, categorical = True, augment = False)

#MODEL LOADING

In [ ]:
# Define metrics
METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
] 

svm_model_loaded = tf.keras.models.load_model("svm_all_data.h5")
svm_model_loaded.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  metrics=METRICS)

#LIME EXPLAINABILITY

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
def getSVMProbabilities(img):
  predictions = svm_model_loaded.predict(img) # one-vs-all scores
  predictions = np.exp(predictions)/np.sum(np.exp(predictions),axis=1, keepdims=True) # softmax after the scoring
  return predictions

#NORMAL

In [ ]:
iterator = iter(test_gen)
images, labels = next(iterator)

In [ ]:
temp, mask = exp.get_image_and_mask(exp.top_labels[0], positive_only=True, num_features=5, hide_rest=True)

plt.imshow(mark_boundaries(temp, mask, color=(1, 0, 0)))

In [ ]:
def show_xai_results(image, path):
  exp = explainer.explain_instance(image, 
                                 getSVMProbabilities, 
                                 top_labels=3, 
                                 hide_color=0, 
                                 num_samples=1000,
                                 random_seed = SEED)
  temp, mask = exp.get_image_and_mask(exp.top_labels[0], positive_only=False, num_features=5, hide_rest=False)

  #Select the same class explained on the figures above.
  ind =  exp.top_labels[0]

  #Map each explanation weight to the corresponding superpixel
  dict_heatmap = dict(exp.local_exp[ind])
  heatmap = np.vectorize(dict_heatmap.get)(exp.segments) 

  fig, axis = plt.subplots(1,3, figsize = (10,30))
  axis = axis.flatten()

  axis[0].imshow(image, cmap='gray')
  axis[0].set_axis_off()
  

  axis[1].imshow(mark_boundaries(temp, mask, color=(1, 0, 0)))
  axis[1].set_axis_off()

  aim = axis[2].imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
  axis[2].set_axis_off()

  fig.colorbar(aim, ax=axis[2], fraction=0.046, pad=0.04)

  plt.get_current_fig_manager().full_screen_toggle()
  plt.savefig(path, bbox_inches='tight')
  plt.show()
  

In [ ]:
show_xai_results(images[0], path = 'data_for_report/svm-xai-normal-exp.pdf')

#PNEUMONIA

In [ ]:
show_xai_results(images[6], path = 'data_for_report/svm-xai-pneumonia-hm.pdf')

#Tuberculusis

In [ ]:
show_xai_results(images[9], path = 'data_for_report/svm-xai-tuberculosis-hm.pdf')